# JuSpyce API test: null maps generation & permutation test

In [1]:
import sys
import os
from glob import glob
import pathlib
import numpy as np
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce
from juspyce.stats import *
from juspyce.utils import *

/Users/llotter/projects/juspyce


## Load JuSpyce data from test_juspyce.fit.ipynb

In [2]:
juspyce_vol = JuSpyce.from_pickle(os.path.join(wd, "testing", "test_juspyce_vol.pkl.gz"))

INFO:juspyce.api:Loaded complete object from /Users/llotter/projects/juspyce/testing/test_juspyce_vol.pkl.gz.


## Permutation based on null maps

Uses the method from brainsmash based on variograms to correct for spatial autocorrelation. Distance maps are based on geodesic distances for surface parcellations and euclidean distances for volumetric parcellations.

One has to choose a method for which empirical p-values will be calculated. If the `JuSpyce.permute_maps()` method is then applied again for computation of another metric, the already existing null maps will be used (this behavior can be turned off).

In [3]:
# everything below that is set to None will be set to customs or taken from the .fit() method
_ = juspyce_vol.permute_maps(
    method="dominance", # which method
    comparison=None, # use dataframe from .compare() function
    permute="X", # null maps for which dataset? Can be 'X' or 'Y'
    null_method="random", # "variogram" -> brainsmash, "random" -> np.random
    null_maps=None, # directly provide null maps. Must be dict with keys corresponding to the maps
                    # for which null data is generated (e.g., "mGluR5" or so)
    use_null_maps=True, # re-use null maps if already computed for another prediction method
    dist_mat=None, # custom distance matrix -> must be ndarray with shape (n_parcels, n_parcels) 
                   # or tuple of those ndarrays if parcellation is tuple of giftis
    n_perm=1000, # number of permutations (= number of null maps)
    parcellation=None, parc_space=None, parc_hemi=None, # parcellation data, usually set at .fit()
    centroids=False, # compute distance matrices between parcel centroids? -> faster
    r_to_z=True, adjust_r2=True, mlr_individual=True, # settings for prediction (see other notebook)
    p_tail=None, # which-sided p-value? Must be dict with e.g. 
                 # {"spearman":"two"} or 
                 # {"dominance_total":"upper", "dominance_relative":"upper", 
                 #  "dominance_full_r2":"upper", "dominance_individual":"upper"}, etc.
    n_proc=None, # number of processes
    n_proc_predict=1, # number of processes for prediction method -> multiplies itself with n_proc!
    seed=None, # seed for reproducability
    store=True)

INFO:juspyce.api:Running 'true' prediction (method = 'dominance').


Predicting (dominance, 1 proc):   0%|          | 0/28 [00:00<?, ?it/s]

INFO:juspyce.api:No null maps found.
INFO:juspyce.api:Generating null maps for 'X' data (n = 1000, null_method = 'random').


Generating X null data:   0%|          | 0/7 [00:00<?, ?it/s]

Null predictions (dominance, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = '{'dominance_total': 'upper', 'dominance_individual': 'upper', 'dominance_relative': 'upper', 'dominance_full_r2': 'upper'}').


### Print result

All results are stored in a dict as `JuSpyce.p_predictions["prediction_name"]` which corresponds to `JuSpyce.predictions["prediction_name"]`

In [4]:
for dom in ["dominance_full_r2", "dominance_relative", "dominance_total", "dominance_individual"]:
    print(dom)
    display(juspyce_vol.p_predictions[dom])

dominance_full_r2


,dominance_full_r2
control,0.001
touch,0.001
interoception,0.001
learning,0.001
attention,0.001
language,0.006
interaction,0.001
inhibition,0.001
somatosensory,0.001
decision,0.001


dominance_relative


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.251,0.603,0.612,0.589,0.342,0.565,0.513
touch,0.285,0.587,0.313,0.607,0.560,0.637,0.479
interoception,0.669,0.231,0.239,0.509,0.628,0.645,0.535
learning,0.479,0.571,0.472,0.607,0.521,0.633,0.217
attention,0.621,0.505,0.634,0.150,0.579,0.652,0.555
language,0.134,0.649,0.564,0.675,0.526,0.602,0.669
interaction,0.583,0.612,0.542,0.125,0.629,0.658,0.636
inhibition,0.230,0.602,0.593,0.588,0.392,0.619,0.572
somatosensory,0.326,0.593,0.263,0.602,0.573,0.645,0.480
decision,0.607,0.600,0.592,0.514,0.546,0.374,0.242


dominance_total


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.001,0.157,0.059,0.060,0.001,0.123,0.010
touch,0.005,0.108,0.003,0.140,0.056,0.301,0.018
interoception,0.318,0.002,0.006,0.049,0.256,0.380,0.061
learning,0.001,0.002,0.001,0.056,0.003,0.264,0.001
attention,0.336,0.014,0.250,0.001,0.182,0.465,0.066
language,0.001,0.402,0.104,0.447,0.083,0.204,0.484
interaction,0.110,0.154,0.060,0.001,0.228,0.545,0.341
inhibition,0.001,0.085,0.032,0.104,0.001,0.066,0.013
somatosensory,0.005,0.090,0.001,0.148,0.063,0.346,0.018
decision,0.037,0.064,0.063,0.001,0.001,0.001,0.001


dominance_individual


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.001,0.410,0.017,0.372,0.001,0.739,0.859
touch,0.045,0.125,0.183,0.310,0.367,0.218,0.134
interoception,0.308,0.001,0.003,0.004,0.838,0.393,0.016
learning,0.001,0.001,0.001,0.003,0.001,0.655,0.001
attention,0.098,0.001,0.484,0.001,0.174,0.538,0.002
language,0.001,0.389,0.058,0.696,0.247,0.132,0.578
interaction,0.185,0.954,0.013,0.001,0.816,0.480,0.455
inhibition,0.001,0.218,0.353,0.525,0.001,0.303,0.987
somatosensory,0.080,0.086,0.072,0.303,0.595,0.280,0.099
decision,0.001,0.017,0.094,0.001,0.001,0.001,0.001


## Correct p-values

p values can be corrected across dataframes or rows/columns of dataframes using `JuSpyce.correct_p()`. The method will, if not provided differently, loop over all p-value dataframes and apply multiple comparison correction methods from `statsmodels.stats.multitest.multipletests`.

Results will be stored in the `JuSpyce.p_predictions` dict as `JuSpyce.p_predictions["prediction_name--correction_method"]`, e.g., if method is `spearman` and correction is `fdr_bh:` `juspyce_vol.p_predictions["spearman--fdr_bh"]`

In [6]:
juspyce_vol.correct_p(
    analysis="predictions", # one of "predictions" or "comparisons" -> here: predictions
    method="all", # if all, iterate over all dataframes (but calculate values for each individual dataframe)
    mc_alpha=0.05, # alpha treshold, should have no effect
    mc_method="fdr_bh", # correction method passed to statsmodels
    mc_dimension="array") # 'array', 'row' or 'column'
display(juspyce_vol.p_predictions["dominance_total--fdr_bh"])

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.005026,0.210767,0.109688,0.109688,0.005026,0.179910,0.028406
touch,0.017818,0.161588,0.011529,0.197410,0.105538,0.349089,0.044658
interoception,0.361410,0.008340,0.019600,0.097010,0.304097,0.404783,0.109688
learning,0.005026,0.008340,0.005026,0.105538,0.011529,0.309844,0.005026
attention,0.375295,0.037081,0.300613,0.005026,0.230142,0.484787,0.112487
language,0.005026,0.425903,0.158016,0.471032,0.136706,0.253063,0.499284
interaction,0.163333,0.208166,0.109688,0.005026,0.279300,0.553472,0.377605
inhibition,0.005026,0.137686,0.072092,0.158016,0.005026,0.112487,0.034904
somatosensory,0.017818,0.142258,0.005026,0.205730,0.110250,0.380989,0.044658
decision,0.078826,0.111009,0.110250,0.005026,0.005026,0.005026,0.005026


## Use "comparison" Y data

In [7]:
# groups
n_Y = juspyce_vol.Y.shape[0]
groups = [0] * int(n_Y/2) + [1] * int(n_Y/2)
print(n_Y, groups)

28 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
# apply comparison
juspyce_vol.compare(
    comparison="diff(mean(A),mean(B))",
    groups=groups
)
# run prediction & permutation
_ = juspyce_vol.permute_maps(
    method="spearman",
    comparison="diff(mean(A),mean(B))", 
    permute="X",
    n_perm=1000,
    store=True)

INFO:juspyce.api:Subtracting parcelwise mean of B from mean of A: new Y = mean(Y[A]) - mean(Y[B]).
INFO:juspyce.api:Running 'true' prediction (method = 'spearman').


Predicting (spearman, 1 proc):   0%|          | 0/1 [00:00<?, ?it/s]

Null predictions (spearman, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = '{'spearman': 'two'}').


In [9]:
juspyce_vol.p_predictions["diff(mean(A),mean(B))-spearman"]

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.001,0.178,0.498,0.006,0.004,0.468,0.338


In [11]:
juspyce_vol.correct_p(analysis="predictions")
display(juspyce_vol.p_predictions["diff(mean(A),mean(B))-spearman--fdr_bh"])

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.007,0.3115,0.498,0.014,0.014,0.498,0.4732
